Esmalt loeb failid listi ning viib nad paremini käideltatavale kujule

In [1]:
from estnltk import Text
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup as bs

sample_data = ["eesti_ilu", "eesti_seadus", "luup"]

def read_files_from_dir(dir_name):
    total_data = ""

    for f in listdir(dir_name):
        if isfile(join(dir_name, f)):
            total_data += formatted_data(join(dir_name, f))
        else:
            total_data += read_files_from_dir_with_cache(f)

    return total_data

def formatted_data(filename):
    with open(filename, "r") as file:
        content = "".join(file.readlines())
        bs_content = bs(content, "lxml")
        return bs_content.text

texts = [Text(read_files_from_dir(x)).tag_analysis() for x in sample_data]

KeyboardInterrupt: 

In [2]:
from collections import Counter

semantics = ["holonyms", "hypernyms", "synonyms"]

connections = dict()

for semantic in semantics:
    local = dict()
    connections[semantic] = local

    with open("noun_" + semantic + "_estwn_2.3.2.txt", "r", encoding="utf-8") as f:
        for line in f.read().split("\n"):
            if line:
                
                line = line.replace("<", "|")
                x, y = line.split("|")

                if not local.get(x):
                    local[x] = set()
                local[x].add(y)
                
                if not local.get(y):
                    local[y] = set()
                local[y].add(x)

In [3]:
lemmas = dict()  # cache for lemmas

In [4]:
section_counters = [
    dict(),
    dict(),
    dict()
]

for i in range(3):
    for semantic in semantics:
        section_counters[i][semantic] = [0, Counter()]

for i, text in enumerate(texts):
    local_counter = section_counters[i]
    for semantic in semantics:
        graph = connections[semantic]

        for i, word in enumerate(text.words):
            if word["text"] not in lemmas:
                lemmas[word["text"]] = word["analysis"][0]["lemma"]

            if lemmas[word["text"]] in graph:
                dist = " "

                for di in range(1, 5):
                    try:
                        other_word = text.words[i + di]
                    except:
                        break

                    if not other_word["text"] in lemmas:
                        lemmas[other_word["text"]] = other_word["analysis"][0]["lemma"]

                    if lemmas[other_word["text"]] in graph:
                        local_counter[semantic][0] += 1
                        local_counter[semantic][1][dist] += 1
                        break
                    else:
                        dist += other_word["text"] + " "

In [5]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

table = []
uniques = set()

for text in range(3):
    for semantic in semantics:
        total, counter = section_counters[text][semantic]
        for item, count in counter.most_common(10):
            uniques.add(item)
            table.append([
                sample_data[text] + " " + semantic,
                "S" + item + "S",
                count,
                count / total
            ])


pd.DataFrame(table, columns=["source", "item", "count", "contribution to corpus"])

,source,item,count,contribution to corpus
0,eesti_ilu holonyms,S S,61818,0.245673
1,eesti_ilu holonyms,"S , S",7482,0.029734
2,eesti_ilu holonyms,S . S,7245,0.028793
3,eesti_ilu holonyms,S ja S,4899,0.019469
4,eesti_ilu holonyms,S on S,1528,0.006072
5,eesti_ilu holonyms,"S . "" S",1106,0.004395
6,eesti_ilu holonyms,S oma S,934,0.003712
7,eesti_ilu holonyms,S oli S,921,0.003660
8,eesti_ilu holonyms,"S , et S",857,0.003406
9,eesti_ilu holonyms,S ta S,804,0.003195


In [6]:
table = []

for text in range(3):
    for unique in uniques:
        counts = []
        tot_counts = []
        for semantic in semantics:
            tot, counter = section_counters[text][semantic]
            counts.append(counter[unique])
            tot_counts.append(tot)
      
        pre = sum(counts)
        if pre == 0:
            counts.append(1000000)

        row = [
            sample_data[text],
            "S" + unique + "S", pre,
            counts[0] / sum(counts),
            counts[0] / sum(tot_counts),
            counts[1] / sum(counts),
            counts[1] / sum(tot_counts),
            counts[2] / sum(counts),
            counts[2] / sum(tot_counts)
        ]
        table.append(row)

pd.DataFrame(table, columns=[
    "source",
    "item",
    "count",
    "holonym % types",
    "holonym % corpus",
    "hypernym % types",
    "hypernym % corpus",
    "synonym % types",
    "synonym % corpus"
])

,source,item,count,holonym % types,holonym % corpus,hypernym % types,hypernym % corpus,synonym % types,synonym % corpus
0,eesti_ilu,"S , et S",8980,0.095434,3.062808e-04,0.581403,1.865918e-03,0.323163,1.037138e-03
1,eesti_ilu,S oma S,934,1.000000,3.337996e-04,0.000000,0.000000e+00,0.000000,0.000000e+00
2,eesti_ilu,S ; S,1725,0.086957,5.360807e-05,0.574493,3.541707e-04,0.338551,2.087141e-04
3,eesti_ilu,S ( 1 ) S,0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
4,eesti_ilu,S . ( S,208,0.067308,5.003420e-06,0.649038,4.824727e-05,0.283654,2.108584e-05
5,eesti_ilu,S S,1066138,0.057983,2.209296e-02,0.581631,2.216154e-01,0.360386,1.373157e-01
6,eesti_ilu,S või S,2671,0.159491,1.522469e-04,0.000000,0.000000e+00,0.840509,8.023342e-04
7,eesti_ilu,S käesoleva S,8,0.125000,3.573872e-07,0.625000,1.786936e-06,0.250000,7.147743e-07
8,eesti_ilu,S oli S,11637,0.079144,3.291536e-04,0.597663,2.485628e-03,0.323193,1.344133e-03
9,eesti_ilu,S ! S,4490,0.079733,1.279446e-04,0.590646,9.477907e-04,0.329621,5.289330e-04


header: holonym % types näitab, et kui suure osa moodustas antud semantiline seos kõikidest semantilistest seosetest

holonym % corpus näiteks näitab, et kui suure osa korpusest leitud mustritest moodustas antud muster. 

Ehk: holonym % types + hypernym % types + synonym % types = 1

Kui arv läheneb ühele, siis saab mustrit kasutada kindla semantilise seose leidmisel

Hinnang:

---

Lõppkokkuvõteks
- "S oma S" mustriga saab hästi ära tuvastaad holonüüme
- "S ja S" mustriga holonüüme
- "S või S" mustriga sünonüüme

Erinevad semantilised seosed andsid sarnaseid tulemusi erinevate tekstikorpustega

Ei pannud kõiki tekstikorpuseid kokku algselt, kuna hüpotees oli, et annavad erinevaid tulemusi. Aga tuleb välja, et annavad sarnaseid.

Kõige sagedasem oli hüperonüümi mustri kohtamine, mis on ilmselt tingitud sellest, et hüperonüüme oli 10 korda rohkem kui teisi
